In [1]:
cd C:\\Users\\Sharuka Thirimanne\\Desktop\\FYP-ML-IDS

C:\Users\Sharuka Thirimanne\Desktop\FYP-ML-IDS


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#### CorrelationAttributeEval function in Weka was used to perform feature selection
Correlation is more formally referred to as Pearson’s correlation coefficient in statistics.

You can calculate the correlation between each attribute and the output variable and select only those attributes that have a moderate-to-high positive or negative correlation (close to -1 or 1) and drop those attributes with a low correlation (value close to zero).

Weka supports correlation based feature selection with the CorrelationAttributeEval technique that requires use of a Ranker search method.

[Link]('https://machinelearningmastery.com/perform-feature-selection-machine-learning-data-weka/#:~:text=Weka%20supports%20correlation%20based%20feature,correlation%20with%20the%20output%20class.')

In [3]:
df = pd.read_csv('Datasets/UNSW_NB15_training-set.csv')
df.drop(['id'], axis = 'columns' , inplace = True)
#df.drop(['attack_cat'], axis = 'columns' , inplace = True)


df.drop(['attack_cat','sbytes', 'sjit', 'sloss', 'dinpkt', 'is_ftp_login', 'ct_ftp_cmd', 
         'ct_src_ltm', 'response_body_len', 'spkts', 'djit','dtcpb',
         'stcpb','ct_dst_ltm','dbytes'], axis = 'columns' , inplace = True)


print('Number of duplicated initially :',df.duplicated().sum())
df.drop_duplicates(keep='first',inplace=True)
print('Number of duplicated after :',df.duplicated().sum())


Number of duplicated initially : 76463
Number of duplicated after : 0


In [4]:
df.columns

Index(['dur', 'proto', 'service', 'state', 'dpkts', 'rate', 'sttl', 'dttl',
       'sload', 'dload', 'dloss', 'sinpkt', 'swin', 'dwin', 'tcprtt', 'synack',
       'ackdat', 'smean', 'dmean', 'trans_depth', 'ct_srv_src', 'ct_state_ttl',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
       'ct_flw_http_mthd', 'ct_srv_dst', 'is_sm_ips_ports', 'label'],
      dtype='object')

In [5]:
df.head()

,dur,proto,service,state,dpkts,rate,sttl,dttl,sload,dload,...,trans_depth,ct_srv_src,ct_state_ttl,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,ct_flw_http_mthd,ct_srv_dst,is_sm_ips_ports,label
0,0.121478,tcp,-,FIN,4,74.087490,252,254,14158.942380,8495.365234,...,0,1,0,1,1,1,0,1,0,0
1,0.649902,tcp,-,FIN,38,78.473372,62,252,8395.112305,503571.312500,...,0,43,1,1,1,2,0,6,0,0
2,1.623129,tcp,-,FIN,16,14.170161,62,252,1572.271851,60929.230470,...,0,7,1,1,1,3,0,6,0,0
3,1.681642,tcp,ftp,FIN,12,13.677108,62,252,2740.178955,3358.622070,...,0,1,1,1,1,3,0,1,0,0
4,0.449454,tcp,-,FIN,6,33.373826,254,252,8561.499023,3987.059814,...,0,43,1,2,1,40,0,39,0,0


In [6]:
df.drop_duplicates(subset=['dur', 'proto', 'service', 'state', 'dpkts', 'rate', 'sttl', 'dttl',
                           'sload', 'dload', 'dloss', 'sinpkt', 'swin', 'dwin', 'tcprtt', 'synack',
                           'ackdat', 'smean', 'dmean', 'trans_depth', 'ct_srv_src', 'ct_state_ttl',
                           'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
                           'ct_flw_http_mthd', 'ct_srv_dst', 'is_sm_ips_ports'],inplace=True)
print('Number of duplicated final :',df.duplicated().sum())

Number of duplicated final : 0


In [7]:
df.shape

(98592, 29)

In [8]:
df['proto'].value_counts()[:8]

tcp     75978
udp     20860
arp       443
unas      273
ospf      190
sctp       60
any        18
igmp       16
Name: proto, dtype: int64

In [9]:
for col in ['proto','service','state']:
        df[col] = df[col].astype('category')

In [10]:
label1 = 'other_proto'
others2 = df['proto'].value_counts().index[2:]
# apply new category label
df['proto'] = df['proto'].cat.add_categories([label1])
df['proto'] = df['proto'].replace(others2, label1)

In [11]:
df['proto'].value_counts()

tcp            75978
udp            20860
other_proto     1754
Name: proto, dtype: int64

In [12]:
df['state'].value_counts()

FIN    74298
CON    12183
INT    11370
REQ      645
RST       83
ECO       10
no         1
URN        1
PAR        1
Name: state, dtype: int64

In [13]:
label2 = 'other_state'
others3 = df['state'].value_counts().index[3:]
# apply new category label
df['state'] = df['state'].cat.add_categories([label2])
df['state'] = df['state'].replace(others3, label2)

In [14]:
df['state'].value_counts()

FIN            74298
CON            12183
INT            11370
other_state      741
Name: state, dtype: int64

In [15]:
df['service'].value_counts()

-           56823
http        17973
dns         10231
smtp         5019
ftp-data     3282
ftp          2689
ssh          1295
pop3         1104
ssl            56
snmp           46
dhcp           37
irc            25
radius         12
Name: service, dtype: int64

In [16]:
label3 = 'other_service'
others_service = df['service'].value_counts().index[6:]
# apply new category label
df['service'] = df['service'].cat.add_categories([label3])
df['service'] = df['service'].replace(others_service, label3)

In [17]:
df['service'].value_counts()

-                56823
http             17973
dns              10231
smtp              5019
ftp-data          3282
ftp               2689
other_service     2575
Name: service, dtype: int64

In [18]:
print('Number of duplicated before :',df.duplicated().sum())
df.drop_duplicates(keep='first',inplace=True)
print('Number of duplicated after :',df.duplicated().sum())

Number of duplicated before : 524
Number of duplicated after : 0


In [38]:
df1 = pd.DataFrame(df.groupby(['attack_cat'])['attack_cat'].count())
df2 = pd.DataFrame(df.groupby(['label'])['label'].count())


KeyError: 'attack_cat'

In [ ]:
def autolabel1(rects,ax):
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2., 1.01*height,
                '%d' % int(height),ha='center', va='bottom',fontsize=10)


In [ ]:
attack_cat = []
label = []
plot = []
attack_cat_labels = ['Analysis','Backdoor','DoS','Exploits','Fuzzers','Generic','Normal','Reconnaissance','Shellcode','Worms']
class_labels = ['Normal','Attack']

for i in range(0,len(df1['attack_cat'])):
    attack_cat.append(df1['attack_cat'].values[i])
for i in range(0,len(df2['label'])):
    label.append(df2['label'].values[i])

fig , (ax1,ax2) = plt.subplots(1,2,figsize=(18,8))
    
pl1 = ax1.bar(np.arange(len(df1['attack_cat'])), attack_cat)
ax1.set_xticks(np.arange(len(attack_cat_labels)))
ax1.set_xticklabels(labels=attack_cat_labels,rotation=45,ha='right')
ax1.set_yticks([])
ax1.set_title('Attack Type count in the Dataset',fontsize=10)
autolabel1(pl1,ax1)

pl2 = ax2.bar(np.arange(len(df2['label'])), label)
ax2.set_xticks(np.arange(len(class_labels)))
ax2.set_xticklabels(labels=class_labels,rotation=45,ha='right')
ax2.set_yticks([])
ax2.set_title('Number of Attacks and Normal Instances in the Dataset',fontsize=10)
autolabel1(pl2,ax2)



In [ ]:
df.groupby('attack_cat')['attack_cat'].count()

In [ ]:
n_proto = []
a_proto = []
n_service = []
a_service = []
n_state = []
a_state = []
n_attack_cat = []
a_attack_cat = []


for i in range(0,7):
    f_death.append(df_new['Deaths'].values[i])
    l_death.append(df_new['Deaths'].values[i+7])
    
for i in range(0,7):
    f_injured.append(df_new['Injured'].values[i])
    l_injured.append(df_new['Injured'].values[i+7])
    
for i in range(0,7):
    f_h_destoyed.append(df_new['Houses Destroyed'].values[i])
    l_h_destoyed.append(df_new['Houses Destroyed'].values[i+7])
    
for i in range(0,7):
    f_h_damaged.append(df_new['Houses Damaged'].values[i])
    l_h_damaged.append(df_new['Houses Damaged'].values[i+7])
    
lst1 = [f_death,f_injured,f_h_destoyed,f_h_damaged]
lst2 = [l_death,l_injured,l_h_destoyed,l_h_damaged]

width =0.3
plots1=[]
plots2=[]
names = ['Deaths','Injured','Houses Destroyed','Houses Damaged']
provinces = ['Central','North-Central','North-Western','Sabaragamuwa','Southern','Uva','Western']

fig, axes = plt.subplots(2,2,figsize=(10,10))  
fig.subplots_adjust(hspace=0.5)

axes = axes.flatten()

for i,ax in enumerate(axes):
    plots1.append(axes[i].bar(np.arange(len(lst1[i])), lst1[i], width=width,label='Flood'))
    plots2.append(axes[i].bar(np.arange(len(lst2[i]))+ width, lst2[i], width=width , label='Landslide'))
    axes[i].set_xticks(np.arange(len(lst[0])) + width / 2)
    axes[i].set_xticklabels(labels = provinces, fontsize = 10, rotation = 45)
    axes[i].legend(loc='best')
    axes[i].set_yticks([])
    axes[i].set_title('Resuls of Floods and Landslides : {}'.format(names[i]),fontsize=10)
    autolabel1(plots1[i],axes[i])
    autolabel1(plots2[i],axes[i])

In [19]:
df_test = pd.read_csv('Datasets/UNSW_NB15_testing-set.csv')
df_test.drop(['id'], axis = 'columns' , inplace = True)
#df_test.drop(['attack_cat'], axis = 'columns', inplace=True)

df_test.drop(['attack_cat','sbytes', 'sjit', 'sloss', 'dinpkt', 'is_ftp_login', 'ct_ftp_cmd', 
         'ct_src_ltm', 'response_body_len', 'spkts', 'djit','dtcpb',
         'stcpb','ct_dst_ltm','dbytes'], axis = 'columns' , inplace = True)

print('Number of duplicated initially :',df_test.duplicated().sum())
df_test.drop_duplicates(keep='first',inplace=True)
print('Number of duplicated after :',df_test.duplicated().sum())

Number of duplicated initially : 29844
Number of duplicated after : 0


In [20]:
df_test.columns

Index(['dur', 'proto', 'service', 'state', 'dpkts', 'rate', 'sttl', 'dttl',
       'sload', 'dload', 'dloss', 'sinpkt', 'swin', 'dwin', 'tcprtt', 'synack',
       'ackdat', 'smean', 'dmean', 'trans_depth', 'ct_srv_src', 'ct_state_ttl',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
       'ct_flw_http_mthd', 'ct_srv_dst', 'is_sm_ips_ports', 'label'],
      dtype='object')

In [21]:
df_test.drop_duplicates(subset=['dur', 'proto', 'service', 'state', 'dpkts', 'rate', 'sttl', 'dttl',
                                'sload', 'dload', 'dloss', 'sinpkt', 'swin', 'dwin', 'tcprtt', 'synack',
                                'ackdat', 'smean', 'dmean', 'trans_depth', 'ct_srv_src', 'ct_state_ttl',
                                'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
                                'ct_flw_http_mthd', 'ct_srv_dst', 'is_sm_ips_ports'],inplace=True)
print('Number of duplicated final :',df_test.duplicated().sum())

Number of duplicated final : 0


In [22]:
df_test.shape

(52464, 29)

In [23]:
df_test['proto'].value_counts()[:8]

tcp     40183
udp     11496
arp       212
unas      170
ospf       72
sctp       26
igmp       26
any         6
Name: proto, dtype: int64

In [24]:
for col in ['proto','service','state']:
        df_test[col] = df_test[col].astype('category')

In [25]:
label1 = 'other_proto'
others4 = df_test['proto'].value_counts().index[2:]
# apply new category label
df_test['proto'] = df_test['proto'].cat.add_categories([label1])
df_test['proto'] = df_test['proto'].replace(others4, label1)

In [26]:
df_test['proto'].value_counts()

tcp            40183
udp            11496
other_proto      785
Name: proto, dtype: int64

In [27]:
label2 = 'other_state'
others5 = df_test['state'].value_counts().index[3:]
# apply new category label
df_test['state'] = df_test['state'].cat.add_categories([label2])
df_test['state'] = df_test['state'].replace(others5, label2)

In [28]:
df_test['state'].value_counts()

FIN            37308
INT             7625
CON             6648
other_state      883
Name: state, dtype: int64

In [29]:
df_test['service'].value_counts()

-           34151
http         7887
dns          5536
smtp         1759
ftp          1290
ftp-data     1177
pop3          381
ssh           204
ssl            30
snmp           22
dhcp           16
radius          6
irc             5
Name: service, dtype: int64

In [30]:
label3 = 'other_service'
others_service1 = df_test['service'].value_counts().index[6:]
# apply new category label
df_test['service'] = df_test['service'].cat.add_categories([label3])
df_test['service'] = df_test['service'].replace(others_service1, label3)
df_test['service'].value_counts()

-                34151
http              7887
dns               5536
smtp              1759
ftp               1290
ftp-data          1177
other_service      664
Name: service, dtype: int64

In [31]:
print('Number of duplicated before :',df_test.duplicated().sum())
df_test.drop_duplicates(keep='first',inplace=True)
print('Number of duplicated after :',df_test.duplicated().sum())

Number of duplicated before : 117
Number of duplicated after : 0


In [32]:
df_new = pd.concat([df,df_test],axis=0)
df_new.shape

(150415, 29)

In [33]:
df_new.drop_duplicates(subset=['dur', 'proto', 'service', 'state', 'dpkts', 'rate', 'sttl', 'dttl',
                               'sload', 'dload', 'dloss', 'sinpkt', 'swin', 'dwin', 'tcprtt', 'synack',
                               'ackdat', 'smean', 'dmean', 'trans_depth', 'ct_srv_src', 'ct_state_ttl',
                               'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
                               'ct_flw_http_mthd', 'ct_srv_dst', 'is_sm_ips_ports'],inplace=True)


In [34]:
df_new.duplicated().sum()

0

In [35]:
df_new.shape

(148985, 29)

In [36]:
df_new.columns

Index(['dur', 'proto', 'service', 'state', 'dpkts', 'rate', 'sttl', 'dttl',
       'sload', 'dload', 'dloss', 'sinpkt', 'swin', 'dwin', 'tcprtt', 'synack',
       'ackdat', 'smean', 'dmean', 'trans_depth', 'ct_srv_src', 'ct_state_ttl',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
       'ct_flw_http_mthd', 'ct_srv_dst', 'is_sm_ips_ports', 'label'],
      dtype='object')

In [37]:
limit = df_new.shape[1]-1
    
X = df_new.iloc[:,0:limit] # train set features
Y = df_new.iloc[:,limit]

In [38]:
from sklearn.model_selection import train_test_split

In [39]:
train_X, test_X , train_Y , test_Y = train_test_split(X, Y, test_size=0.2,stratify=Y,shuffle=True,random_state=5)

print('Training X :',train_X.shape)
print('Test_X :',test_X.shape)
print('Training_Y :',train_Y.shape)
print('Test_Y :',test_Y.shape)

Training X : (119188, 28)
Test_X : (29797, 28)
Training_Y : (119188,)
Test_Y : (29797,)


In [40]:
type(train_X)

pandas.core.frame.DataFrame

In [41]:
df_train = pd.concat([train_X,train_Y],axis=1)
df_train

,dur,proto,service,state,dpkts,rate,sttl,dttl,sload,dload,...,trans_depth,ct_srv_src,ct_state_ttl,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,ct_flw_http_mthd,ct_srv_dst,is_sm_ips_ports,label
46515,0.644512,tcp,-,FIN,8,26.376545,254,252,6.305546e+03,3.847873e+03,...,0,1,1,1,1,1,0,1,0,1
17824,0.006075,tcp,ftp-data,FIN,6,3127.572055,31,29,1.091819e+07,3.516049e+05,...,0,3,0,1,1,5,0,3,0,0
24117,0.011907,tcp,-,FIN,36,5794.910511,31,29,1.454606e+06,9.794575e+06,...,0,3,0,1,1,1,0,6,0,0
649,0.207650,tcp,http,FIN,12,101.131709,62,252,2.839393e+04,3.225812e+05,...,1,2,1,1,1,1,1,1,0,1
78491,0.905537,tcp,-,FIN,8,18.773391,254,252,4.487945e+03,2.738706e+03,...,0,5,1,1,1,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30857,0.016296,tcp,-,FIN,42,4970.545136,31,29,1.216986e+06,1.126608e+07,...,0,10,0,1,1,1,0,4,0,0
83703,0.404591,tcp,http,FIN,6,37.074478,254,252,1.498798e+04,4.429165e+03,...,0,1,1,1,1,1,0,1,0,1
14093,13.278695,tcp,-,FIN,438,50.080222,31,29,7.883305e+03,3.295294e+05,...,0,2,0,2,1,5,0,4,0,0
1279,0.661048,tcp,-,FIN,8,25.716741,254,252,6.147814e+03,3.751619e+03,...,0,1,1,1,1,1,0,1,0,1


In [42]:
df_train.columns

Index(['dur', 'proto', 'service', 'state', 'dpkts', 'rate', 'sttl', 'dttl',
       'sload', 'dload', 'dloss', 'sinpkt', 'swin', 'dwin', 'tcprtt', 'synack',
       'ackdat', 'smean', 'dmean', 'trans_depth', 'ct_srv_src', 'ct_state_ttl',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
       'ct_flw_http_mthd', 'ct_srv_dst', 'is_sm_ips_ports', 'label'],
      dtype='object')

In [43]:
# Class count
count_class_0, count_class_1 = df_train['label'].value_counts()

# Divide by class
df_class_0 = df_train[df_train['label'] == 0]
df_class_1 = df_train[df_train['label'] == 1]
    
df_class_0_under = df_class_0.sample(count_class_1)
df_train = pd.concat([df_class_0_under, df_class_1], axis=0)

In [44]:
df_train.duplicated().sum()

0

In [45]:
df_train.groupby('label')['label'].count()

label
0    52030
1    52030
Name: label, dtype: int64

In [46]:
df_test.groupby('label')['label'].count()

label
0    33494
1    18853
Name: label, dtype: int64

In [47]:
df_train

,dur,proto,service,state,dpkts,rate,sttl,dttl,sload,dload,...,trans_depth,ct_srv_src,ct_state_ttl,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,ct_flw_http_mthd,ct_srv_dst,is_sm_ips_ports,label
1111,2.728969,tcp,-,FIN,18,12.092478,31,29,4.233100e+03,4.552635e+03,...,0,14,0,13,1,4,0,16,0,0
77729,0.921654,tcp,-,FIN,8,18.445100,254,252,1.259475e+04,2.690815e+03,...,0,5,1,1,1,5,0,5,0,0
40223,0.103981,tcp,-,FIN,74,1394.485489,31,29,3.215972e+05,4.828498e+06,...,0,4,0,1,1,4,0,9,0,0
67343,1.681813,tcp,-,FIN,8,10.108139,254,252,2.288007e+03,1.474599e+03,...,0,4,1,1,1,2,0,4,0,0
31168,0.001134,udp,dns,CON,2,2645.502599,31,29,5.149912e+05,6.278659e+05,...,0,3,0,1,1,1,0,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16683,0.000008,udp,dns,INT,0,125000.000300,254,0,5.700000e+07,0.000000e+00,...,0,28,2,3,3,28,0,28,0,1
75602,0.523856,tcp,smtp,FIN,22,112.626374,62,252,4.603402e+05,2.790080e+04,...,0,1,1,1,1,1,0,1,0,1
104376,1.193694,tcp,-,FIN,8,14.241506,254,252,4.671214e+03,2.077584e+03,...,0,10,1,1,1,8,0,6,0,1
83703,0.404591,tcp,http,FIN,6,37.074478,254,252,1.498798e+04,4.429165e+03,...,0,1,1,1,1,1,0,1,0,1


In [48]:
df_train.reset_index(drop=True,inplace=True)
df_train.reset_index(inplace=True)


In [49]:
df_train.to_csv('USNW_train-set.csv')
df_train.head()

,index,dur,proto,service,state,dpkts,rate,sttl,dttl,sload,...,trans_depth,ct_srv_src,ct_state_ttl,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,ct_flw_http_mthd,ct_srv_dst,is_sm_ips_ports,label
0,0,2.728969,tcp,-,FIN,18,12.092478,31,29,4233.100098,...,0,14,0,13,1,4,0,16,0,0
1,1,0.921654,tcp,-,FIN,8,18.445100,254,252,12594.748050,...,0,5,1,1,1,5,0,5,0,0
2,2,0.103981,tcp,-,FIN,74,1394.485489,31,29,321597.218800,...,0,4,0,1,1,4,0,9,0,0
3,3,1.681813,tcp,-,FIN,8,10.108139,254,252,2288.007080,...,0,4,1,1,1,2,0,4,0,0
4,4,0.001134,udp,dns,CON,2,2645.502599,31,29,514991.187500,...,0,3,0,1,1,1,0,3,0,0


In [50]:
df_test = pd.concat([test_X,test_Y],axis=1)
df_test.reset_index(drop=True,inplace=True)
df_test.reset_index(inplace=True)
df_test.to_csv('USNW_test-set.csv')
df_test.head()

,index,dur,proto,service,state,dpkts,rate,sttl,dttl,sload,...,trans_depth,ct_srv_src,ct_state_ttl,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,ct_flw_http_mthd,ct_srv_dst,is_sm_ips_ports,label
0,0,0.079968,tcp,-,FIN,92,2263.405420,31,29,511904.781300,...,0,5,0,1,1,1,0,6,0,0
1,1,0.001068,udp,dns,CON,2,2808.988808,31,29,546816.500000,...,0,4,0,2,1,1,0,3,0,0
2,2,1.614434,tcp,http,FIN,112,88.575934,62,252,15316.823240,...,1,1,1,1,1,1,1,1,0,1
3,3,0.001054,udp,dns,CON,2,2846.299829,31,29,554079.687500,...,0,2,0,2,1,1,0,4,0,0
4,4,2.232915,tcp,http,FIN,10,8.509057,62,252,2568.839355,...,1,1,1,1,1,1,1,1,0,1


In [51]:
df_test.duplicated().sum()

0

In [52]:
print('Train',len(df_train['proto'].unique()))
print('Test',len(df_test['proto'].unique()))
print('=======================================')
print('Train',len(df_train['service'].unique()))
print('Test',len(df_test['service'].unique()))
print('=======================================')
print('Train',len(df_train['state'].unique()))
print('Test',len(df_test['state'].unique()))

Train 3
Test 3
Train 7
Test 7
Train 4
Test 4


In [53]:
df_train['proto'].value_counts()

tcp            81341
udp            21547
other_proto     1172
Name: proto, dtype: int64

In [54]:
df_test['proto'].value_counts()

tcp            23205
udp             6267
other_proto      325
Name: proto, dtype: int64